In [43]:




import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import time
import os
import cv2
import tensorflow as tf
import matplotlib.image as mpimg
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import fetch_olivetti_faces
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from skimage.color import rgb2gray
from tqdm.notebook import tqdm
from mpl_toolkits.mplot3d import Axes3D
from tensorflow.keras import layers, models
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

gpus = tf.config.experimental.list_physical_devices('GPU') #with import tensorflow
tf.config.experimental.set_memory_growth(gpus[0], True) # me too

# 1. 데이터셋 만들기
# 2. 데이터셋 나눠서 훈련 및 테스트
# 3. 모델 저장
# 4. 별도 파일에서 모델 불러오기 후 카메라 이미지판별
# 
#

path = "../cv_data/figure/" ##
label = os.listdir(path)

seed = 13
model = tf.keras.models.load_model("../../SCT_model.h5")




In [63]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 100)               51300     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                6464      
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                        

In [45]:
test_df = pd.DataFrame({"file":os.listdir(path + "test_data")})
test_df["label"] = test_df["file"].apply(lambda x: x.split(".")[0])##



In [64]:
test_df

,file,label
0,star.5.png,star
1,star.42.png,star
2,star.34.png,star
3,circle.13.png,circle
4,star.20.png,star
...,...,...
142,circle.2.png,circle
143,star.2.png,star
144,star.40.png,star
145,star.21.png,star


In [47]:
def resize_and_preprocess_vgg(image):
    image = cv2.resize(image, dsize=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    return preprocess_input(image)

In [89]:

#def predict_vgg(model, image):
image = cv2.imread("../cv_data/figure/test_data/triangle.13.png")
image = resize_and_preprocess_vgg(image)

yhat = model.predict(image)
print(np.argmax(yhat[0]))

1/1 [==============================] - 0s 18ms/step
2


In [90]:
labels = ["circle", "star", "triangle"]
False_img = []

count = 0
path = "../cv_data/figure/test_data/"
img_list = os.listdir(path)
for name in img_list:
    count += 1
    image = cv2.imread(path + name)
    image = resize_and_preprocess_vgg(image)
    result = np.argmax(model.predict(image)[0])

    label = name.split(".")[0]
    label = labels.index(label)
    print(result, label)

    if label != result:
        False_img.append(name)



    


1/1 [==============================] - 0s 16ms/step
1 1
1/1 [==============================] - 0s 16ms/step
1 1
1/1 [==============================] - 0s 14ms/step
1 1
1/1 [==============================] - 0s 31ms/step
0 0
1/1 [==============================] - 0s 23ms/step
1 1
1/1 [==============================] - 0s 23ms/step
2 2
1/1 [==============================] - 0s 27ms/step
2 2
1/1 [==============================] - 0s 20ms/step
2 1
1/1 [==============================] - 0s 20ms/step
0 0
1/1 [==============================] - 0s 19ms/step
0 0
1/1 [==============================] - 0s 18ms/step
1 1
1/1 [==============================] - 0s 17ms/step
2 1
1/1 [==============================] - 0s 20ms/step
0 0
1/1 [==============================] - 0s 21ms/step
2 1
1/1 [==============================] - 0s 22ms/step
0 0
1/1 [==============================] - 0s 18ms/step
2 2
1/1 [==============================] - 0s 16ms/step
2 2
1/1 [==============================] - 0s 21ms/s

In [93]:
print(1 - len(False_img) / len(img_list))

0.9455782312925171
